In [1]:
import numpy as np
import pandas as pd

In [35]:
# トロピカル半環の元
class Tropical:
    def __init__(self, value):
        self.value = value

    # oplusの定義
    def __add__(self, other):
        return Tropical(min(self.value, other.value))
    
    # otimesの定義
    def __mul__(self, other):
        return Tropical(self.value + other.value)
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    # printに対する返り値
    def __str__(self):
        return str(self.value)
    
    # 変数のみの実行に対する返り値
    def __repr__(self):
        return "Tropical : " + str(self.value)

# トロピカルベクトルの定義
class Tropical_Vector:
    def __init__(self, value):
        self.N = len(value)
        self.value = [None] * self.N
        for i in range(self.N):
            self.value[i] = value[i]
            
    # oplusの定義
    def __add__(self, other):
        return Tropical_Vector([Tropical(self.value[i]) + Tropical(other.value[i]) for i in range(self.N)])
    
    # otimesの定義
    def __mul__(self, other):
        if type(other) == Tropical_Vector:
            return Tropical_Vector([Tropical(self.value[i]) * Tropical(other.value[i]) for i in range(self.N)])
        else:
            return Tropical_Vector([Tropical(other)*Tropical(self.value[i]) for i in range(self.N)])
        
    def __rmul__(self, other):
        return self.__mul__(other)
        
        
    # 内積の定義
    def dot(self,other):
        inner_product = Tropical(float('inf'))
        for i in range(self.N):
            inner_product = inner_product + Tropical(self.value[i]) * Tropical(other.value[i])
        return inner_product
        
    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return "Tropical vector : " + str(self.value)
    
    def __getitem__(self, i):
        return self.value[i]
    
# トロピカル行列の定義
class Tropical_Matrix:
    
    # 正方行列のみ
    def __init__(self, value):
        self.N = len(value) # 危険
        self.value = [[None]*self.N for i in range(self.N)]
        for i in range(self.N):
            for j in range(self.N):
                self.value[i][j] = value[i][j]
    
    # oplusの定義
    def __add__(self, other):
        A = [[None]*self.N for i in range(self.N)]
        
        for i in range(self.N):
            for j in range(self.N):
                A[i][j] = (Tropical(self.value[i][j]) + Tropical(other.value[i][j])).value
        return Tropical_Matrix(A)
    
    # otimesの定義
    def __mul__(self, other):
        if type(other) == Tropical_Matrix:
            A = [[None]*self.N for i in range(self.N)]
            for i in range(self.N):
                for j in range(self.N):
                    A[i][j] = (Tropical_Vector(self.value[i]).dot(Tropical_Vector(other.T().value[j]))).value
            return Tropical_Matrix(A)
                                                                  
        else:
            return Tropical_Matrix([[Tropical(other)*self.value[i][j] for i in range(self.N)] for j in range(self.N)])
    
    
    # べき乗の実装
    def __pow__(self, other):
        n = int(other)
        if n == 1:
            return self
        else:
            # トロピカル単位行列
            A = [[float('inf')]*self.N for i in range(self.N)]
            for i in range(self.N):
                A[i][i] = 0
                
            Atp = Tropical_Matrix(A)
            
            for i in range(n):
                Atp = Atp*self
        return Atp
        
    # 転置    
    def T(self):
        A = [[None]*self.N for i in range(self.N)]
        for i in range(self.N):
            for j in range(self.N):
                A[i][j] = self.value[j][i] 
        return Tropical_Matrix(A)   
    
    def __str__(self):
        return str(self.value)
    
    def __repr__(self):
        return "Tropical matrix : " + str(self.value)
    
    # スライス
    def __getitem__(self,key):
        return self.value[key[0]][key[1]]

In [36]:
# 隣接行列の作成
N = 4
A = [[float('inf')]*N for i in range(N)]

A[0][1] = 3
A[0][2] = 5
A[1][2] = 1
A[1][3] = 6
A[2][3] = 3

# トロピカル行列へ変換
Atp = Tropical_Matrix(A)

In [43]:
pd.DataFrame((Atp).value)

,0,1,2,3
0,inf,3.000000,5.000000,inf
1,inf,inf,1.000000,6.000000
2,inf,inf,inf,3.000000
3,inf,inf,inf,inf


In [44]:
pd.DataFrame((Atp**2).value)

,0,1,2,3
0,inf,inf,4.000000,8.000000
1,inf,inf,inf,4.000000
2,inf,inf,inf,inf
3,inf,inf,inf,inf


In [45]:
pd.DataFrame((Atp**3).value)

,0,1,2,3
0,inf,inf,inf,7.000000
1,inf,inf,inf,inf
2,inf,inf,inf,inf
3,inf,inf,inf,inf


In [46]:
pd.DataFrame((Atp**4).value)

,0,1,2,3
0,inf,inf,inf,inf
1,inf,inf,inf,inf
2,inf,inf,inf,inf
3,inf,inf,inf,inf


In [47]:
pd.DataFrame((Atp**5).value)

,0,1,2,3
0,inf,inf,inf,inf
1,inf,inf,inf,inf
2,inf,inf,inf,inf
3,inf,inf,inf,inf


In [49]:
pd.DataFrame((Atp + Atp**2+Atp**3).value)

,0,1,2,3
0,inf,3.000000,4.000000,7.000000
1,inf,inf,1.000000,4.000000
2,inf,inf,inf,3.000000
3,inf,inf,inf,inf
